In [ ]:
#!python
from libs import *
from helper_functions import *
from paths import *
%matplotlib inline

In [ ]:
# gauth = GoogleAuth(settings_file=code_ / "config/settings.yaml")           
# drive = GoogleDrive(gauth)
# minute_folder = '1tc6yb3kkqmgyDh7am7SU17_FupTlBg9o'

In [ ]:
nifty500_str =  nifty500_txt.read_text()
os.chdir(minute_nifty500)
nifty500_list = tv_str_to_list(nifty500_str)

In [ ]:
start_ = '2018-01-01'
end_ = '2022-12-31'

DOWNLOADING NIFTY500 DATA

In [ ]:
# file_list = drive.ListFile({'q': "'{}' in parents and trashed=false".format(minute_folder)}).GetList()

In [ ]:
# e = 0
# for i, file in enumerate(sorted(file_list, key = lambda x: x['title']), start=1):
#     title = file['title']
#     print(i, title)
#     symbol = title.split('_')[0]
#     if symbol in nifty500_list:
#         try:
#             print('Downloading {} file from GDrive ({}/{})'.format(file['title'], i, len(file_list)))
#             file.GetContentFile(file['title'])
#         except:
#             e += 1
#             print('Error downloading symbol: ', symbol)
#     # break

In [ ]:
def vwap(df):
    q = df.volume.values
    p = df.close.values
    return df.assign(vwap=(p * q).cumsum() / q.cumsum())
def typical_vwap(df):
    q = df.volume.values
    p = df.typical_price.values
    return df.assign(typical_vwap=(p * q).cumsum() / q.cumsum())

In [ ]:
files = glob('*2018*.csv')

In [ ]:
for f in files:
    df = pd.read_csv(f)
    # try:
    if 'typical_price' not in df.columns:
        df['typical_price'] = (df['high']+df['low']+ df['close'])/3
    if 'typical_vwap' not in df.columns:
        df = df.groupby(df.Date, group_keys=False).apply(typical_vwap)
    if 'vwap_close' not in df.columns:
        df['vwap_close'] = df['vwap'] - df['close']
    if 'vwap_typical' not in df.columns:
        df['vwap_typical'] = df['vwap'] - df['typical_price']
    for i in df.columns:
        if 'Unnamed' in i:
            df.drop(i, inplace = True, axis=1)
    df.to_csv(f)
    print(f)
# except:
    #     print('Error in Symbol', f)

In [ ]:
def check_vols(df):
    if max(df.volume) <= 0:
        return False    
    return True

In [ ]:
for f in files:
    try:
        df = pd.read_csv(f)
        for name, group in df.groupby(['Date']):
            # print(group.shape[0])
            if not check_vols(group) or group.shape[0] < 375:
                idx = df[df['Date'] == name].index
                df.drop(idx, inplace=True)
        df.to_csv(f.replace('.csv', '_filtered.csv'))
        print(f)
    except:
        print('Error in ', f)

4. MOTIF SEARCH

In [ ]:
for f in files:
    try:
        df = pd.read_csv(f)
        for name, group in df.groupby(['Date']):
            if not check_vols(group) or group.shape[0] < 360:
                indexAge = df[df['Date'] == name].index
                df.drop(indexAge , inplace=True)
        df.to_csv(f)
        print(f)
    except:
        print('Error in ', f)

In [ ]:
tf = 60
Ts = []

In [ ]:
for f in files:
    print(f)
    df = pd.read_csv(f)
    for name, group in df.groupby(['Date']):
        # open_ = df['open'].iloc[0]
        # close_ = df['close'].iloc[-1]
        Ts.append(group['vwap_close'])

In [ ]:
m = tf  # Chosen since the eog signal was downsampled to 50 Hz
radius, Ts_idx, subseq_idx = stumpy.ostinato(Ts, m)

In [ ]:
print(f'Found Best Radius {np.round(radius, 2)} in time series {Ts_idx} starting at subsequence index location {subseq_idx}.')

In [ ]:
consensus_motif = Ts[Ts_idx][subseq_idx : subseq_idx + m]
nn_idx = []
for i, T in enumerate(Ts):
    nn_idx.append(np.argmin(stumpy.core.mass(consensus_motif, T)))
    lw = 1
    label = None
    if i == Ts_idx:
        lw = 4
        label = 'Consensus Motif'
    plt.plot(stumpy.core.z_norm(T[nn_idx[i] : nn_idx[i]+m]), lw=lw, label=label)

plt.title('The Consensus Motif (Z-normalized)')
plt.xlabel('Time (s)')
plt.legend()
plt.show()